In [1]:
# 分析MovieLens 电影分类中的频繁项集和关联规则
import pandas as pd
from  mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

#数据加载
movies = pd.read_csv('./movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [2]:
# 将genres进行one-hot编码（离散特征有多少取值，就用多少维来表示这个特征）
movies_hot_encoded = movies.drop('genres',1).join(movies.genres.str.get_dummies(sep='|'))
pd.options.display.max_columns=100
movies_hot_encoded.head()

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
# 将movieId, title设置为index
movies_hot_encoded.set_index(['movieId','title'],inplace=True)
# 挖掘频繁项集，最小支持度为0.02
itemsets = apriori(movies_hot_encoded,use_colnames=True,min_support=0.02)
# 按照支持度从大到小进行时候粗
itemsets = itemsets.sort_values(by='support',ascending=False)
print('-'*20,'频繁项集','-'*20)
print(itemsets)

-------------------- 频繁项集 --------------------
     support                  itemsets
7   0.489185                   (Drama)
4   0.306987                  (Comedy)
14  0.153164                (Thriller)
12  0.151294                 (Romance)
0   0.129042                  (Action)
5   0.107743                   (Crime)
9   0.095718                  (Horror)
31  0.094325          (Drama, Romance)
26  0.093335           (Drama, Comedy)
6   0.090586             (Documentary)
1   0.085380               (Adventure)
27  0.069470         (Romance, Comedy)
32  0.068480         (Drama, Thriller)
13  0.063898                  (Sci-Fi)
28  0.062761            (Drama, Crime)
11  0.055503                 (Mystery)
8   0.051763                 (Fantasy)
29  0.045165         (Thriller, Crime)
20  0.044101           (Drama, Action)
15  0.043772                     (War)
3   0.041755                (Children)
22  0.040655        (Thriller, Action)
34  0.039336        (Thriller, Horror)
10  0.037979     

In [4]:
# 根据频繁项集计算关联规则，设置最小提升度为2
rules = association_rules(itemsets,metric='lift',min_threshold=0.1)
# 按照提升度从大到小进行排序
rules = rules.sort_values(by='lift',ascending=False)
print('-'*20,'关联规则','-'*20)
print(rules)

-------------------- 关联规则 --------------------
          antecedents        consequents  antecedent support  \
23          (Mystery)         (Thriller)            0.055503   
22         (Thriller)          (Mystery)            0.153164   
45            (Crime)  (Drama, Thriller)            0.107743   
40  (Drama, Thriller)            (Crime)            0.068480   
19        (Adventure)           (Action)            0.085380   
18           (Action)        (Adventure)            0.129042   
47           (Action)           (Sci-Fi)            0.129042   
46           (Sci-Fi)           (Action)            0.063898   
10         (Thriller)            (Crime)            0.153164   
11            (Crime)         (Thriller)            0.107743   
16         (Thriller)           (Horror)            0.153164   
17           (Horror)         (Thriller)            0.095718   
41     (Drama, Crime)         (Thriller)            0.062761   
44         (Thriller)     (Drama, Crime)            0.153